In [388]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.
INFO: Found ChipWhisperer😍


In [389]:
scope.clock.clkgen_freq=200E6

In [390]:
scope.glitch.clk_src="clkgen"

In [391]:
scope.glitch.resetDCMs()

In [392]:
scope.glitch.output="enable_only"

In [393]:
#Reset chip - this is NEEDED for device to start.
#Also you can reset it 


import os

check_died = 0

reset_num = 0

def rest_power():
    global check_died,reset_num
    # 方法一：使用智能插座开关
    os.system(r"C:\platform-tools\adb.exe shell input tap 900 800")
    time.sleep(2)
    os.system(r"C:\platform-tools\adb.exe shell input tap 900 800")

    #等待系统完全启动
    time.sleep(40)
    check_died = 0
    reset_num = reset_num + 1
    print(reset_num)
    # 方法二：使用继电器开关
    # scope.io.tio3 = False
    # time.sleep(0.05)
    # scope.io.tio3 = True
    


In [394]:
def rest_login():
    target.flush()
    target.baud = 115200
    target.write("\n")
    target.write("pi\n")
    time.sleep(0.8)
    target.write("123456\n")
    time.sleep(0.8)
    target.write("./test\n")
    

In [395]:
glitch_num = 0

def check_success():
    global check_died,  glitch_num
    val = target.simpleserial_read_witherrors('R', 25, end = '\n', glitch_timeout=50)#For loop check

    valid = val['valid']
    # if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']


    if "Result2" in raw_serial:
        if "-5894076284012091139" not in raw_serial:
            print("success")
            print(raw_serial)
            print(scope.glitch.repeat)
            check_died = 0
            return True
             
        else:
            check_died = 0
            print("Glitching...")
            print(raw_serial)
            
            glitch_num = glitch_num + 1
            print(f"glitch_num: {glitch_num}")
            
            return False
     
    if "Result2" not in raw_serial:
            check_died = check_died + 1
            if check_died > 10:
                check_died = 0
                glitch_num = 0
                print(f"check_died: {check_died}")
                print("resert system")
                rest_power()
                rest_login()
                
    target.flush()
    

In [396]:
scope.io.glitch_hp=True
scope.glitch.width = 20

In [ ]:
import time 
import random

rest_power()
rest_login()
time.sleep(1)

# scope.glitch.repeat = 3400  #  in home

scope.glitch.repeat = 3460  #  in home

while True:
     
    scope.glitch.manual_trigger()
    # 生成一个0.1到0.5之间的随机数
    sleep_time = random.uniform(0.1, 0.5)
    # 暂停相应的时间
    time.sleep(sleep_time)


    val =  check_success()

    if val == True:
        print(scope.glitch.repeat)
        break

    if glitch_num > 120:
        glitch_num = 0
        scope.glitch.repeat =  scope.glitch.repeat + 10
        rest_power()
        rest_login()
        time.sleep(1)
            
print("Done glitching")


1
Glitching...
Result1: -5894076284012091139, Result2: 1

glitch_num: 1
Glitching...
Result1: -5894076284012091139, Result2: 2

glitch_num: 2
Glitching...
Result1: -5894076284012091139, Result2: 3

glitch_num: 3
Glitching...
Result1: -5894076284012091139, Result2: 4

glitch_num: 4
Glitching...
Result1: -5894076284012091139, Result2: 5

glitch_num: 5
Glitching...
Result1: -5894076284012091139, Result2: 6

glitch_num: 6
Glitching...
Result1: -5894076284012091139, Result2: 7

glitch_num: 7
Glitching...
Result1: -5894076284012091139, Result2: 8

glitch_num: 8
Glitching...
Result1: -5894076284012091139, Result2: 9

glitch_num: 9
Glitching...
Result1: -5894076284012091139, Result2: 10

glitch_num: 10
Glitching...
Result1: -5894076284012091139, Result2: 11

glitch_num: 11
Glitching...
Result1: -5894076284012091139, Result2: 12

glitch_num: 12
Glitching...
Result1: -5894076284012091139, Result2: 13

glitch_num: 13
Glitching...
Result1: -5894076284012091139, Result2: 14

glitch_num: 14
Glitchin

Glitching...
Result1: -5894076284012091139, Result2: 118

glitch_num: 115
Glitching...
Result1: -5894076284012091139, Result2: 119

glitch_num: 116
Glitching...
Result1: -5894076284012091139, Result2: 120

glitch_num: 117
Glitching...
Result1: -5894076284012091139, Result2: 121

glitch_num: 118
Glitching...
Result1: -5894076284012091139, Result2: 122

glitch_num: 119
Glitching...
Result1: -5894076284012091139, Result2: 123

glitch_num: 120
Glitching...
Result1: -5894076284012091139, Result2: 124

glitch_num: 121
2
Glitching...
Result1: -5894076284012091139, Result2: 1

glitch_num: 1
Glitching...
Result1: -5894076284012091139, Result2: 2

glitch_num: 2
Glitching...
Result1: -5894076284012091139, Result2: 3

glitch_num: 3
Glitching...
Result1: -5894076284012091139, Result2: 4

glitch_num: 4
Glitching...
Result1: -5894076284012091139, Result2: 5

glitch_num: 5
Glitching...
Result1: -5894076284012091139, Result2: 6

glitch_num: 6
Glitching...
Result1: -5894076284012091139, Result2: 7

glit

Glitching...
Result1: -5894076284012091139, Result2: 111

glitch_num: 108
Glitching...
Result1: -5894076284012091139, Result2: 112

glitch_num: 109
Glitching...
Result1: -5894076284012091139, Result2: 113

glitch_num: 110
Glitching...
Result1: -5894076284012091139, Result2: 114

glitch_num: 111
Glitching...
Result1: -5894076284012091139, Result2: 115

glitch_num: 112
Glitching...
Result1: -5894076284012091139, Result2: 116

glitch_num: 113
Glitching...
Result1: -5894076284012091139, Result2: 117

glitch_num: 114
Glitching...
Result1: -5894076284012091139, Result2: 118

glitch_num: 115
Glitching...
Result1: -5894076284012091139, Result2: 120

glitch_num: 116
Glitching...
Result1: -5894076284012091139, Result2: 121

glitch_num: 117
Glitching...
Result1: -5894076284012091139, Result2: 122

glitch_num: 118
Glitching...
Result1: -5894076284012091139, Result2: 123

glitch_num: 119
Glitching...
Result1: -5894076284012091139, Result2: 124

glitch_num: 120
Glitching...
Result1: -589407628401209

Glitching...
Result1: -5894076284012091139, Result2: 105

glitch_num: 101
Glitching...
Result1: -5894076284012091139, Result2: 106

glitch_num: 102
Glitching...
Result1: -5894076284012091139, Result2: 107

glitch_num: 103
Glitching...
Result1: -5894076284012091139, Result2: 108

glitch_num: 104
Glitching...
Result1: -5894076284012091139, Result2: 109

glitch_num: 105
Glitching...
Result1: -5894076284012091139, Result2: 110

glitch_num: 106
Glitching...
Result1: -5894076284012091139, Result2: 111

glitch_num: 107
Glitching...
Result1: -5894076284012091139, Result2: 112

glitch_num: 108
Glitching...
Result1: -5894076284012091139, Result2: 113

glitch_num: 109
Glitching...
Result1: -5894076284012091139, Result2: 114

glitch_num: 110
Glitching...
Result1: -5894076284012091139, Result2: 115

glitch_num: 111
Glitching...
Result1: -5894076284012091139, Result2: 116

glitch_num: 112
Glitching...
Result1: -5894076284012091139, Result2: 117

glitch_num: 113
Glitching...
Result1: -589407628401209

Glitching...
Result1: -5894076284012091139, Result2: 97

glitch_num: 94
Glitching...
Result1: -5894076284012091139, Result2: 98

glitch_num: 95
Glitching...
Result1: -5894076284012091139, Result2: 99

glitch_num: 96
Glitching...
Result1: -5894076284012091139, Result2: 100

glitch_num: 97
Glitching...
Result1: -5894076284012091139, Result2: 101

glitch_num: 98
Glitching...
Result1: -5894076284012091139, Result2: 102

glitch_num: 99
Glitching...
Result1: -5894076284012091139, Result2: 103

glitch_num: 100
Glitching...
Result1: -5894076284012091139, Result2: 104

glitch_num: 101
Glitching...
Result1: -5894076284012091139, Result2: 105

glitch_num: 102
Glitching...
Result1: -5894076284012091139, Result2: 106

glitch_num: 103
Glitching...
Result1: -5894076284012091139, Result2: 107

glitch_num: 104
Glitching...
Result1: -5894076284012091139, Result2: 108

glitch_num: 105
Glitching...
Result1: -5894076284012091139, Result2: 109

glitch_num: 106
Glitching...
Result1: -5894076284012091139, Res

Glitching...
Result1: -5894076284012091139, Result2: 89

glitch_num: 87
Glitching...
Result1: -5894076284012091139, Result2: 90

glitch_num: 88
Glitching...
Result1: -5894076284012091139, Result2: 91

glitch_num: 89
Glitching...
Result1: -5894076284012091139, Result2: 92

glitch_num: 90
Glitching...
Result1: -5894076284012091139, Result2: 93

glitch_num: 91
Glitching...
Result1: -5894076284012091139, Result2: 94

glitch_num: 92
Glitching...
Result1: -5894076284012091139, Result2: 95

glitch_num: 93
Glitching...
Result1: -5894076284012091139, Result2: 96

glitch_num: 94
Glitching...
Result1: -5894076284012091139, Result2: 97

glitch_num: 95
Glitching...
Result1: -5894076284012091139, Result2: 98

glitch_num: 96
Glitching...
Result1: -5894076284012091139, Result2: 99

glitch_num: 97
Glitching...
Result1: -5894076284012091139, Result2: 100

glitch_num: 98
Glitching...
Result1: -5894076284012091139, Result2: 101

glitch_num: 99
Glitching...
Result1: -5894076284012091139, Result2: 102

glit

Glitching...
Result1: -5894076284012091139, Result2: 83

glitch_num: 80
Glitching...
Result1: -5894076284012091139, Result2: 84

glitch_num: 81
Glitching...
Result1: -5894076284012091139, Result2: 85

glitch_num: 82
Glitching...
Result1: -5894076284012091139, Result2: 86

glitch_num: 83
Glitching...
Result1: -5894076284012091139, Result2: 87

glitch_num: 84
Glitching...
Result1: -5894076284012091139, Result2: 88

glitch_num: 85
Glitching...
Result1: -5894076284012091139, Result2: 89

glitch_num: 86
Glitching...
Result1: -5894076284012091139, Result2: 90

glitch_num: 87
Glitching...
Result1: -5894076284012091139, Result2: 91

glitch_num: 88
Glitching...
Result1: -5894076284012091139, Result2: 92

glitch_num: 89
Glitching...
Result1: -5894076284012091139, Result2: 93

glitch_num: 90
Glitching...
Result1: -5894076284012091139, Result2: 94

glitch_num: 91
Glitching...
Result1: -5894076284012091139, Result2: 95

glitch_num: 92
Glitching...
Result1: -5894076284012091139, Result2: 96

glitch_

Glitching...
Result1: -5894076284012091139, Result2: 75

glitch_num: 73
Glitching...
Result1: -5894076284012091139, Result2: 76

glitch_num: 74
Glitching...
Result1: -5894076284012091139, Result2: 77

glitch_num: 75
Glitching...
Result1: -5894076284012091139, Result2: 78

glitch_num: 76
Glitching...
Result1: -5894076284012091139, Result2: 79

glitch_num: 77
Glitching...
Result1: -5894076284012091139, Result2: 80

glitch_num: 78
Glitching...
Result1: -5894076284012091139, Result2: 81

glitch_num: 79
Glitching...
Result1: -5894076284012091139, Result2: 82

glitch_num: 80
Glitching...
Result1: -5894076284012091139, Result2: 83

glitch_num: 81
Glitching...
Result1: -5894076284012091139, Result2: 84

glitch_num: 82
Glitching...
Result1: -5894076284012091139, Result2: 85

glitch_num: 83
Glitching...
Result1: -5894076284012091139, Result2: 86

glitch_num: 84
Glitching...
Result1: -5894076284012091139, Result2: 87

glitch_num: 85
Glitching...
Result1: -5894076284012091139, Result2: 88

glitch_